# Machine Learning Interatomic Potentials for Li-ion Diffusion in Batteries

**AMET Computational Methods for Energy Materials Practical**

---

In this practical, we will work through the full pipeline of training machine-learned interatomic potentials (MLIPs) on Density Functional Theory (DFT) data, to then run fast molecular dynamics (MD) simulations to study ion transport in a solid-state electrolyte:

1. **Explore** a dataset of DFT calculations for a lithium thiophosphate (Li-P-S) system
2. **Train** a MLIP on the DFT data, and **validate** its accuracy
3. **Run** molecular dynamics (MD) simulations with the trained ML potential
4. **Extract** the Li-ion diffusion coefficient from the MD trajectories
5. **Determine** the activation energy for Li-ion diffusion, via the Arrhenius relation

### Why does this matter?

Solid-state electrolytes are of great interest for next-generation lithium batteries with no liquid electrolyte components, offering improved safety (no more YouTube clips of Tesla batteries catching fire) and energy density.

Understanding Li-ion transport at the atomic scale requires long molecular dynamics simulations that are often too expensive with DFT (AIMD) alone. MLIPs bridge this gap: we train a ML model on a relatively small set of DFT data, which we can afford to compute, then use this ML potential to run simulations orders of magnitude faster — at near-DFT accuracy.

**Note:**

In this practical there are a number of technical and discussion/conceptual questions. It is recommended that you record answers as you go, and download the completed notebook at the end of the practical session, to use as a resource for reference with the practical report and follow-on task.

## Setup and Installation

First, let's install the required packages. This practical uses:
- **[MACE](https://github.com/ACEsuit/mace)**: A state-of-the-art equivariant MLIP architecture
- **[ASE](https://wiki.fysik.dtu.dk/ase/)**: The Atomic Simulation Environment for structure manipulation and MD
- Standard scientific Python: `numpy`, `matplotlib`, `scipy`

As discussed in our lectures, GPUs can be much faster for Molecular Dynamics and ML training/deployment. We can use GPUs in Colab by following these instructions:

**To activate the GPU, click on the down arrow in the top right of the editor, click on "Change runtime type" and select "T4 GPU" from the menu Hardware accelerator and save.
Then click on "Connect". If this doesn't work, change all "cuda" instances in the notebook to "cpu" (and use the slower CPU option instead).**

In [ ]:
# this automatically downloads our data, so we don't need to manually download and upload
# as we did in the practical session:
!git clone https://github.com/SAM-Laboratory/AMET_Comp_Practical
%cd AMET_Comp_Practical

In [ ]:
# Install required packages:
!pip install mace-torch ase matplotlib numpy scipy pymatviz pymatgen
!pip install cuequivariance_torch  # this speeds up our model training (accelerated tensor products)
# Make sure Runtime -> Change runtime type -> GPU is selected

In [ ]:
# import standard scientific Python packages; numpy, matplotlib:
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter("ignore")  # ignore irrelevant warnings for this practical

---
# Part 1: Exploring the DFT Training Data

Before training any model, we need to understand our reference data. We have a dataset of DFT calculations for a lithium thiophosphate system stored in `LiPS.xyz.gz`.

**Key questions to answer:**
- What are the chemical compositions present in the dataset?
- How large are the simulation cells?
- What is the range of energies and forces?
- What do the structures look like?
- How many configurations are there? (i.e. how much data do we have?)
- How many atoms of each _element_ do we have in the dataset?

In [ ]:
!gzip -d LiPS.xyz.gz  # decompress the data file

Here the data is stored in XYZ format (.xyz). Let's quickly look at the first few lines of the file to show what this format looks like:

In [ ]:
!head LiPS.xyz  # using "!<command...>" runs a shell command in Jupyter notebooks

In [ ]:
# show the lines 80-90:
!head -n 90 LiPS.xyz | tail -n 10

We can see that the data file contains information about the number of atoms, the lattice parameters, the energy of the cell, then lines with the atomic species, positions, and forces (in x/y/z axes), for each calculation in the dataset. We typically refer to each set of atomic positions, forces, and corresponding energy as a 'frame' or 'configuration'.

We can use the [`ase.io.read`](https://ase-lib.org/ase/io/io.html) function to load the data into a list of ASE `Atoms` objects, which are Python classes that store information about a given simulation cell (structure, energy, forces, etc.).

Let's load the data and run some simple analyses:

In [ ]:
# Load the dataset, which is stored in XYZ format (.xyz)
# Each frame contains: atomic positions, forces, total energy, and the simulation cell
from ase.io import read

print("Loading DFT dataset...")
dataset = read("LiPS.xyz", index=":")  # read all frames in the dataset
print(f"Loaded {len(dataset)} configurations")

### 1.1 Chemical Composition

Let's determine what elements are present and in what proportions. This will tell us what
materials/compositions we're working with.

In [ ]:
# dataset is a list of Atoms objects
print(type(dataset))

In [ ]:
# let's look at just the first configuration in the dataset (list) to start:
first_frame = dataset[0]
print(first_frame)

We can see that just printing the `Atoms` object shows us information about the configuration, including the chemical species present, the cell parameters, etc.

For instance here we see that the configuration has a chemical formula of Li27P12S44. Can this formula be reduced?

If we haven't used ASE before, how can we figure out what information is stored in the `Atoms` object, and how to access it?

1. Documentation – We can find online documentation for ASE, and the `Atoms` class. See for example the [ASE Atoms class documentation](https://ase-lib.org/ase/atoms.html).
    
    For reasonably popular codes, generative AI tools like ChatGPT can also give useful information, but as always we should be suspicious of the accuracy of the answers! Ask a genAI tool what the `Atoms` class is, and what information it contains.

From looking at the documentation, how can we get information about the chemical composition of the configuration?

In [ ]:
first_frame.get_chemical_symbols()

In [ ]:
first_frame.get_chemical_formula()

2. Docstrings – We can access the 'docstring' (short descriptions of the object/functions), using the `help` function, by looking in the Python code itself, using `<object>?` in a Jupyter notebook cell, or by other means.

In [ ]:
help(first_frame)

In [ ]:
first_frame?


3. `dir()` – We can use the `dir()` function to get a list of all the attributes (information) and methods (functions) of an object.

In [ ]:
dir(first_frame)

Ok here it looks like using the `get_chemical_formula()` function will give us the information we need. We could also use `get_chemical_symbols()` to get the list of elements present.

In [ ]:
first_frame.get_chemical_formula()  # same formula, not reduced

In [ ]:
# just for context, we can also use the pymatgen package (pymatgen.org) to get the reduced chemical formula:
from pymatgen.core.composition import Composition

comp = Composition("Li27P12S44")
print(comp)
comp.get_reduced_formula_and_factor()

Let's check if all the compositions are the same, or if there are different compositions present in the dataset:

In [ ]:
# let's first get a list of all the chemical formulae present in the dataset:
list_of_formulas = []
for frame in dataset:
    list_of_formulas.append(frame.get_chemical_formula())

# alternatively, more concisely, we could use a Python 'list comprehension':
# list_of_formulas = [frame.get_chemical_formula() for frame in dataset]
print(list_of_formulas)

In [ ]:
# this is a very long list; how can we determine the unique formulae?
# we can use a Python set to get the unique formulae:
unique_formulas = list(set(list_of_formulas))
print(unique_formulas)

In [ ]:
# Advanced:
# we could also use a Python dictionary to count the number of times each formula appears:
formula_counts = {}
for formula in list_of_formulas:
    if formula in formula_counts:
        formula_counts[formula] += 1
    else:
        formula_counts[formula] = 1

# or numpy:
# formula_counts = np.unique(list_of_formulas, return_counts=True)
print(formula_counts)

So here we are dealing with a dataset of all Li27P12S44 configurations. This is one of the Li-P-S class of solid-state electrolytes; specifically a Li-deficient derivative of Li7P3S11 (which is known as "LPS"). Some relevant papers about LiPS:
- https://pubs.rsc.org/en/content/articlelanding/2014/ee/c3ee41655k
- https://pubs.acs.org/doi/10.1021/acs.chemmater.6b02163

4 formula units of Li7P3S11 gives Li28P12S44, while we have Li27P12S44 – so what type of 'defect' do we have in our system?

### 1.2 Simulation Cell Sizes

How many atoms in the frames?

In [ ]:
natoms = [len(frame) for frame in dataset]
print(set(natoms))  # many ways of doing this; all the same 83 atoms per frame

What about the cell volumes?

In [ ]:
volumes = [frame.get_volume() for frame in dataset]
print(set(volumes))  # many ways of doing this; all the same (fixed) volume

What cell lengths?

In [ ]:
print(first_frame.cell.lengths())

### 1.3 Energy and Force Distributions

The energy and force distributions give us an indication about the range of configurations sampled by the DFT calculations – or in other words, how much of the potential energy surface (PES) we have explored.

Wide energy/force distributions mean diverse training data (wide exploration), while narrow distributions mean only a narrow region of PES is sampled.

Which is better for 'generalisability' (i.e. how well the model will perform on new structures/configurations)?

Which is better for accuracy in the same region of the PES (if we just need the model to very accurately reproduce DFT for a specific region of the PES)?

How can we access the energy and forces from the `Atoms` object? (Tip: Use the instructions above on how to access information/functions of an object in Python!)

Let's plot the distribution of these energies and forces. First we need to extract the energies and forces:

In [ ]:
# Extract energies and forces from all configurations
# this code needs to be filled in:
energies = ...  # array of shape (nframes,)
energies_per_atom = ...  # array of shape (nframes,)
forces = ... # likely to be array of shape (nframes, natoms, 3) with x/y/z components

Given energies and forces arrays, this code plots the distributions:

In [ ]:
# convert x/y/z force components to total force vector norm
force_magnitudes = np.linalg.norm(forces, axis=-1)  # converts (nframes, natoms, 3) to array of shape (nframes, natoms)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Energy distribution
axes[0].hist(energies, bins=60, color="steelblue", edgecolor="white", alpha=0.8)
axes[0].set_xlabel("Energy (eV)")
axes[0].set_ylabel("Count")
axes[0].set_title("Energy Distribution")

# Energy per atom distribution
axes[1].hist(energies_per_atom, bins=60, color="mediumpurple", edgecolor="white", alpha=0.8)
axes[1].set_xlabel("Energy per atom (eV)")
axes[1].set_ylabel("Count")
axes[1].set_title("Energy Distribution")

# Force magnitude distribution (sample for speed)
sample_forces = force_magnitudes[::10].flatten()  # every 10th frame
axes[2].hist(sample_forces, bins=80, color="coral", edgecolor="white", alpha=0.8, range=(0, 5))
axes[2].set_xlabel("|F| (eV/Å)")
axes[2].set_ylabel("Count")
axes[2].set_title("Force Magnitude Distribution")

plt.tight_layout()
plt.show()

print(f"Energy range: {energies.min():.4f} to {energies.max():.4f} eV; spread: {energies.max() - energies.min():.4f} eV")
print(f"Per-atom energy range: {energies_per_atom.min():.4f} to {energies_per_atom.max():.4f} eV/atom; spread: {energies_per_atom.max() - energies_per_atom.min():.4f} eV/atom")
print(f"Mean |F|: {force_magnitudes.mean():.3f} eV/Å")
print(f"Max |F|: {force_magnitudes.max():.3f} eV/Å")

### 1.3 Visualising the Structure

Let's look at what these structures actually look like. We can use the `plot_atoms` and `view` function from `ase.visualize` to visualise the structure:

In [ ]:
from ase.visualize.plot import plot_atoms

plot_atoms(first_frame, radii=0.8, rotation="10x,10y,0z")

In [ ]:
from ase.visualize import view

view(first_frame, viewer='x3d')
# this plot is interactive! you can drag and rotate the structure here


A more advanced, interactive visualisation can be obtained using `pymatviz` (a recently developed package for visualising and analysing materials data – [LinkedIn post](https://www.linkedin.com/posts/janosh-riebesell_two-updates-to-share-one-personal-and-one-activity-7353429148192653312-3cc3?utm_source=share&utm_medium=member_desktop&rcm=ACoAABqLVRMBp4swyiCmc7OnE6Iy_7fIdPmF_wE)):

In [ ]:
from pymatviz import widgets

widgets.StructureWidget(first_frame)

This widget has a lot of useful tools, including the ability to measure distances and angles, change view orientation, and more.

We can also use the `TrajectoryWidget` to visualise a set of multiple configurations:

In [ ]:
widgets.TrajectoryWidget(dataset[:1000])

Here we can press the play button to iterate through the different structures (frames) in our dataset – hint: increase the frames per second to ~20 for a smoother animation.
Looking at the data this way, we can learn a few things about the dataset:

- What type of calculation(s) did this data come from? e.g. geometry relaxations, random snapshots of different configurations (e.g. from 'rattling'), ab-initio molecular dynamics, volume scaling, etc.

- What can we guess about the simulation conditions? (Ensemble choice, temperature, etc...)

- Which atoms seem to be the most mobile / easily displaced in this structure? Does this make sense? Why?

The energy spread across the dataset is relatively narrow. Why is this? *(Hint: these are all configurations of the same material at similar conditions.)*

---
# Part 2: Training a MACE Model
In this practical, we will fit and test a `MACE` model (Message Passing Neural Network), which is a highly accurate and efficient MLIP (Machine Learned Interatomic Potential). The training/testing techniques we show here, however, are broadly applicable to all MLIPs.

You can independently learn about MACE by studying the [original method paper](https://proceedings.neurips.cc/paper_files/paper/2022/file/4a36c3c51af11ed9f34615b81edb5bbc-Paper-Conference.pdf). MACE was developed by unifying the Atomic Cluster Expansion (ACE) approach with the Neural Equivariant Interatomic Potentials (NequIP). The mathematical formalism which unifies these methods is explained in the [accompaning paper](https://doi.org/10.48550/arXiv.2205.06643). Another [useful reference](https://doi.org/10.48550/arXiv.2305.14247) showcases the method's performance on published benchmark datasets. The [code implementation](https://github.com/ACEsuit/mace) is publically available and [here](https://mace-docs.readthedocs.io/en/latest/) you can find the documentation.

## Learning Objectives for today:

1. **Understanding the data**
2. **Fitting and testing MACE models**
3. **Running Molecular Dynamics**

Here we have a high-level explanation of the important parameters in MACE. You can consult the [documentation](https://github.com/ACEsuit/mace) for additional parameters.

- ##### <font color='red'>--num_interactions</font>: message-passing layers

Controls the number of message-passing layers in our deep neural network model.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <font color='red'>--num_channels=32</font>

Controls the width of our neural network layers.

- ##### <font color='red'>--r_max</font>: the cutoff radius

The radial cut-off applied to the local atomic environment in each layer. `r_max=3.0` means atoms separated by a distance of more than 3.0 A do not directly _communicate_. When the model has multiple message-passing layers, atoms further than 3.0 A can still _communicate_ through later messages if intermediate proxy atoms exist. The effective _receptive field_ of the model is `num_interactions x r_max` (i.e. number of layers times the per-layer radial cutoff).

- ##### <font color='red'>--max_ell</font>: angular resolution

The angular resolution describes how well the model can describe angles.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <font color='red'>--max_L=1</font>

Controls the level of 'equivariance' of the model; L=0 is invariant, L=1 is equivariant, L>=2 are higher levels of equivariance.

<font color='blue'>**In general, the `accuracy` of the model can be improved by using more layers, more channels or higher equivariances. This will result in more parameters and `slower` models, as discussed in lectures.**</font>


Let's train our first model!

### 2.1 Preparing the Data

We split the dataset into training, validation, and test sets. Typically the ML model is trained on the training set, constantly evaluated on the validation set, and then tested/evaluated on the test set at the end of training.

The validation set is used during training to monitor for overfitting (typically when training error is decreasing, but validation error is increasing).
The test set provides an independent measure of accuracy *after* training is complete.

As discussed above, this data comes from ab-initio molecular dynamics simulations, one of the common data generation approaches for MLIPs. However one drawback of this approach is that the data is highly-correlated, with each successive step in the MD simulation being quite similar to the previous step. This increases the redundancy in the data, and so often we sub-sample the full MD trajectory to reduce the amount of data we need to train on while retaining most of the useful information.

Here we'll subsample to retain 10% of the MD trajectory data, and then use a random split of this subsampled data to generate the training, validation, and test sets, which we can do using the `train_test_split` function from `sklearn` (scikit-learn). Let's use a 80:10:10 split for the training, validation, and test sets:

In [ ]:
from ase.io import write
from sklearn.model_selection import train_test_split

# first, subsample 10% of the dataset
subsampled_dataset = dataset[::10]  # this takes every 10th frame in the dataset

# First split off train from test/val; with 80/20 split
train_set, test_val_set = train_test_split(subsampled_dataset, train_size=0.8, test_size=0.2)

# split test/val into val and test; with 50/50 split (corresponding to 10/10 split overall)
test_set, val_set = train_test_split(test_val_set, test_size=0.5, train_size=0.5)

# Save split datasets to files for MLIP training
write("LiPS_train.xyz", train_set)
write("LiPS_val.xyz", val_set)
write("LiPS_test.xyz", test_set)

print(f"Training set:   {len(train_set)} configurations")
print(f"Validation set: {len(val_set)} configurations")
print(f"Test set:       {len(test_set)} configurations")

### 2.2 Training the Model

We write a YAML configuration file and call the MACE training script.
With this initial setup, training should converge in roughly 10 minutes on a GPU.

While the model trains, watch the output — you should see the energy and force errors decreasing with each epoch.

In [ ]:
# Write the MACE configuration file:
%%writefile mace_config.yml
# model hyperparameters
model: "MACE"
num_interactions: 2
num_channels: 32
r_max: 4.0
max_ell: 1
max_L: 0

# training settings
name: "LiPS_MACE"  # name for our model
train_file: "LiPS_train.xyz"
valid_file: "LiPS_val.xyz"
test_file: "LiPS_test.xyz"
device: cuda  # 'cuda' for GPU training, 'cpu' for CPU training
batch_size: 30
max_num_epochs: 50  # ~10 mins with current model, with GPU training
default_dtype: float32
E0s: average
swa: True
# default values are shown here: https://github.com/ACEsuit/mace/blob/main/mace/tools/arg_parser.py

There are some additional settings we must give in our training configuration file:

- ##### <font color='red'>--name</font>: the name of the model
This name will be used to form file names (model, log, checkpoints, results), so choose a distinct name for each experiment.

- ##### <font color='red'>--train/valid/test_file</font>: path to training/validation/testing data

- ##### <font color='red'>--device</font> computing device to use
Can be CPU (`cpu`) or GPU (`cuda`). Here we will use `cuda` since the GPU will be significantly faster than the CPU.

- ##### <font color='red'>--batch_size</font> number of frames to evaluate in one 'batch' (i.e. model parameter update). This training strategy is called stochastic gradient descent because only a subset of the data (`batch_size`) is used to change the parameters at each update.

- ##### <font color='red'>--max_num_epochs</font> maximum number of passes through the data ('epochs'). An _epoch_ is completed when the entire training data has been used once in updating the weights _batch_ by _batch_. 

Now we are ready to fit our first MACE model:

In [ ]:
# Train the MACE model;
# this command prints a lot of information and many warnings (which we can ignore)
# here we have some code which runs the main MACE training command, and tries to minimise the warnings output:
# we don't need to worry about what these individual function calls are doing; main thing to know is that it is
# running MACE model training using the config file (`mace_config.yml`) that we specify
import sys
import logging
from mace.cli.run_train import main as mace_run_train_main

def train_mace(config_file_path):
    """Train a MACE model from a YAML config file."""
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]
    mace_run_train_main()

print("Starting MACE training...")
train_mace("mace_config.yml")
print("Training complete!")

Sometimes Colab times out and cuts access to files in this current 'runtime', so it is recommended to download the `LiPS_MACE_stagetwo_compiled.model` file to your local PC now; by clicking on the 'Files' tab on the left and downloading.

In [ ]:
# Cleanup operations;
# this code just removes some checkpoint files since they may cause errors on retraining a model with the same name but a different architecture:
import glob
import os
for file in glob.glob("checkpoints/*_run-*.model"):
    os.remove(file)
for file in glob.glob("checkpoints/*.pt"):
    os.remove(file)

We could tune the hyperparameters used to train the model to improve its accuracy. For now, we will proceed with the current parameters, but later we will revisit this.

### 2.3 Evaluating the Model

A good MLIP should accurately reproduce the DFT energies and forces. We check this
with **parity plots**: predicted vs. reference values. Points close to the diagonal
line $y = x$ indicate good predictions.

The key metrics are:
- **Energy MAE** (mean absolute error): typically should be ~1–5 meV/atom for a good potential
- **Force MAE**: typically should be <50 meV/Å for reliable MD

To evaluate the model performance, we load the trained model, and use it to predict the energies and forces for our dataset:

In [ ]:
from mace.calculators import MACECalculator

# load our model using the MACECalculator class, which works with ASE:
calc = MACECalculator(
    model_paths=["LiPS_MACE_stagetwo_compiled.model"],  # file path of trained model
    device="cuda",  # change to "cpu" if no CUDA GPU
    default_dtype="float32",
)

# Evaluate on our dataset:
# let's create a dictionary to store the results for each dataset:
dataset_evaluations = {}

from tqdm import tqdm  # we can use this to show a progress bar

for dataset_name in ["train", "val", "test"]:
    data = read(f"LiPS_{dataset_name}.xyz", index=":")  # load the data

    # we will store the results in lists:
    dft_energies = []
    dft_forces = []
    mace_energies = []
    mace_forces = []

    for atoms in tqdm(data):  # iterate over the frames in the data
        # first collect reference DFT energies and forces:
        dft_energies.append(atoms.info["REF_energy"] / len(atoms))  # energies per atom
        dft_forces.append(atoms.arrays["REF_forces"].flatten())  # forces per atom
        # (we use flatten() to convert the 1x3 array of x/y/z forces to a 1D array, that we can add to lists)

        # then predict energies and forces using our trained model:
        # to do this with ASE, we attach our trained model Calculator to the atoms
        # object (atoms.calc), which is then used when we call the
        # get_potential_energy() or get_forces() methods:
        atoms.calc = calc  # set the calculator to our trained model
        mace_energies.append(atoms.get_potential_energy() / len(atoms))  # energies per atom
        mace_forces.append(atoms.get_forces().flatten())  # forces per atom

    # convert the lists to numpy arrays for easier processing, and add to dict:
    dataset_evaluations[dataset_name] = {
        "dft_energies": np.array(dft_energies),
        "dft_forces": np.array(dft_forces),
        "mace_energies": np.array(mace_energies),
        "mace_forces": np.array(mace_forces),
    }

To analyse the model performance, we can generate 'parity plots' of the predicted vs. reference energies and forces:

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 13))

for i, dataset_name in enumerate(["train", "val", "test"]):
    data = dataset_evaluations[dataset_name]
    dft_e = data["dft_energies"]
    ml_e = data["mace_energies"]
    dft_f = data["dft_forces"]
    ml_f = data["mace_forces"]

    # Energy metrics (meV/atom)
    e_mae = np.mean(np.abs(dft_e - ml_e)) * 1000
    e_rmse = np.sqrt(np.mean((dft_e - ml_e) ** 2)) * 1000

    # Force metrics (meV/Å)
    f_mae = np.mean(np.abs(dft_f - ml_f)) * 1000
    f_rmse = np.sqrt(np.mean((dft_f - ml_f) ** 2)) * 1000

    # Energy parity plot
    ax_e = axes[i, 0]
    ax_e.scatter(dft_e, ml_e, s=8, alpha=0.5, color="steelblue",
                 label=f"MAE = {e_mae:.1f} meV/atom\nRMSE = {e_rmse:.1f} meV/atom")
    lims = [min(dft_e.min(), ml_e.min()), max(dft_e.max(), ml_e.max())]
    ax_e.plot(lims, lims, "k--", linewidth=1)
    ax_e.set_xlabel("DFT energy (eV/atom)")
    ax_e.set_ylabel("MACE energy (eV/atom)")
    ax_e.set_title(f"{dataset_name} — Energy")
    ax_e.set_aspect("equal")
    ax_e.legend(fontsize=8, loc="upper left")

    # Force parity plot
    ax_f = axes[i, 1]
    ax_f.scatter(dft_f, ml_f, s=1, alpha=0.1, color="coral",
                 label=f"MAE = {f_mae:.1f} meV/Å\nRMSE = {f_rmse:.1f} meV/Å")
    flims = [dft_f.min(), dft_f.max()]
    ax_f.plot(flims, flims, "k--", linewidth=1)
    ax_f.set_xlabel("DFT force component (eV/Å)")
    ax_f.set_ylabel("MACE force component (eV/Å)")
    ax_f.set_title(f"{dataset_name} — Forces")
    ax_f.set_aspect("equal")
    ax_f.legend(fontsize=8, loc="upper left", markerscale=10)

plt.tight_layout()
plt.show()

### Questions — Part 2

We want to use this model for long-time MD simulations of Li ion diffusion in LiPS. How can we tell if it's accurate enough?

- Which do we think is more important for this task; energy or force accuracy? Why?
- How do the train/val/test MAE/RMSEs compare? Does this give us some indications about overfitting? Based on these metrics, do we think the model is sufficiently accurate?
  - What about the difference in MAE to RMSE? Does this give us some insight about our data/error distribution?
- Typical required force accuracies for ion diffusion MD simulations are ~,50 meV/Å – how does our model compare?
- What are some strategies we could use to further validate the performance of our model (and its predictions in MD simulations)?
- What could we do to improve our model accuracy?
- What would happen if we trained on fewer configurations (e.g., 100 instead of 1000)? What about more (e.g., 5000)?
- What is the effect of the cutoff radius `r_max`? How would increasing it affect accuracy and computational cost? What about other model hyperparameters?

---
# Part 3: Molecular Dynamics Simulations

Now that we have a trained MLIP (i.e. energy & forces calculator), we can use it to run molecular dynamics —
propagating the atomic positions and velocities as a function of time, using classical Newtonian mechanics to generate a trajectory.

### Key MD concepts:

- **Timestep**: Typically should be small enough to resolve the fastest vibrations. For systems with light atoms like Hydrogen, in our lectures we said 0.5–1 fs is typical. What is the lightest atom in our system here, and what timestep do we think we should use?
  - Remember for deciding our timestep; too little and the simulation is inefficient, too large and we will get integration errors and unstable dynamics.
- **MD Ensemble**: The set of thermodynamic variables held constant during the simulation. We will use the same ensemble as the reference data here – was it NVT or NPT?
- **Equilibration**: The initial part of the MD simulation where the system relaxes from the starting configuration to thermal equilibrium. We discard this data.
- **Production**: The part of the MD simulation *after* equilibration, from which we extract physical properties.

### Our plan:
We will run a demonstration MD simulation at 800 K. At elevated temperatures, Li ions
diffuse faster, making it easier to observe diffusion on accessible MD timescales.

### 3.1 Setting Up the Simulation

To start, we will use the same simulation cell as the training data, taking a random configuration from the dataset as our starting structure.
We will initialise atomic velocities from a Maxwell-Boltzmann distribution, corresponding to our simulation temperature of 800 K.

In [ ]:
# here we define a function to run a simple MD simulation, which uses functions from the ASE package:
from ase import units
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

import random
import os
import pylab as pl
from IPython import display

def simpleMD(atoms_init, temp, calc, output_fname, timestep, num_steps):
    # set our calculator:
    atoms_init.set_calculator(calc)

    # initialize the temperature
    random.seed(701)  # (this is just to make the MD simulation fully reproducible)
    MaxwellBoltzmannDistribution(atoms_init, temperature_K=temp)  # initialize temperature at `temp`

    # set up our MD dynamics; here we use a 'Langevin' thermostat,
    # which is a simple way to control the temperature of the system.
    # we also set the timestep here
    dyn = Langevin(atoms_init, timestep*units.fs, temperature_K=temp, friction=0.1)
    # see here: https://ase-lib.org/ase/md.html

    # here we add some code to monitor the temperature and energy of the system as
    # a function of time in the MD simulation:
    time_fs = []
    temperature = []
    energies = []

    os.system('rm -rfv '+output_fname)  # remove any previously stored trajectory with the same name
    fig, ax = pl.subplots(2, 1, figsize=(6,6), sharex='all', gridspec_kw={'hspace': 0, 'wspace': 0}, dpi=500)  # generates dynamic plot as MD proceeds

    def write_frame():  # this is a function we add to the MD dynamics to monitor the system
        dyn.atoms.write(output_fname, append=True)  # store current frame in output trajectory file
        time_fs.append(dyn.get_time()/units.fs)  # store the current time in fs
        temperature.append(dyn.atoms.get_temperature())  # store the current temperature
        energies.append(dyn.atoms.get_potential_energy()/len(dyn.atoms))  # store the current energy

        # plot the energy of the system as a function of time
        ax[0].plot(np.array(time_fs), np.array(energies), color="b")
        ax[0].set_ylabel('E (eV/atom)')

        # plot the temperature of the system as a function of time
        ax[1].plot(np.array(time_fs), temperature, color="r")
        ax[1].set_ylabel('T (K)')
        ax[1].set_xlabel('Time (fs)')

        display.clear_output(wait=True)
        display.display(pl.gcf())

    dyn.attach(write_frame)  # adds our writing & plotting function to the MD calculation
    dyn.run(num_steps)  # runs our MD calculation

In [ ]:
# let's run MD with this function:
atoms_init = dataset[1447].copy()
temp = 550  # in K
timestep = 20  # timestep in fs
num_steps = 500  # max number of steps
output_fname = 'MD_550K.xyz'  # output trajectory file

simpleMD(atoms_init, temp, calc, output_fname, timestep, num_steps)

By monitoring the temperature and energy, we can see if the simulation is proceeding as expected, and when our equilibration phase has completed.

- Is this MD simulation running ok? Why / why not?
    - No, timestep too big, it's exploding!



In [ ]:
# let's run MD with this function:
atoms_init = dataset[1447].copy()
temp = 550  # in K
timestep = 1  # timestep in fs
num_steps = 500  # 5000 steps
output_fname = 'MD_550K.xyz'  # output trajectory file

simpleMD(atoms_init, temp, calc, output_fname, timestep, num_steps)

Our temperature, which is the average kinetic energy of the system, is still varying somewhat here. What could we do to reduce these fluctuations a bit more?

A typical rule of thumb is that for a well-equilibrated NVT simulation, the temperature variation $\Delta T$ should be approximately equal to $T \times \sqrt{2/f}$ where $f = 3N$ is the number of degrees of freedom in the system, with $N$ being the number of particles (atoms). How does our temperature variation here compare to this expectation?​

When we use a MLIP as our energy/forces calculator in MD, we know that there are still some (hopefully small) remnant errors in the energy/forces prediction at each timestep. In short, this means that the energy surface of our calculator is not as 'smooth'. This means that the MD simulation becomes more sensitive to potential sources of numerical errors – what might we want to change as a result, to ensure stable MLIP-accelerated MD simulations?

- When does the equilibration phase complete? How can we tell?

### 3.2 Visualising the Trajectory

We can load the saved trajectory and visualise it, using the same tools we used earlier to visualise the training data. You should be able to see the Li
ions moving through the PS₄ framework — this is ionic diffusion in action!

In [ ]:
# Load our MD trajectory data (from `output_fname` in the simpleMD function above)
# we can do this using the same code we used earlier to read data from the xyz format:
from ase.io import read
traj = read("MD_550K.xyz", index=":")
print(f"Loaded {len(traj)} trajectory frames")

In [ ]:
from ase.visualize.plot import plot_atoms

# let's generate images of a few snapshots from our trajectory:
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
# this selects the 100th, 200th, 300th, 400th frames from the trajectory for plotting
for i, idx in enumerate([100, 200, 300, 400]):  # snapshots at 1,2,3,4 ps
    plot_atoms(traj[idx], axes[i], rotation="10x,10y,0z")
    axes[i].set_title(f"t ≈ {idx/1000:.0f} ps")
    axes[i].axis("off")

plt.suptitle("Trajectory snapshots at 500 K", fontsize=14)
plt.tight_layout()
plt.show()

Using the `TrajectoryWidget` tool:

In [ ]:
widgets.TrajectoryWidget(traj)

**RDF Analysis:**

Let's look at the Radial Distribution Function (RDF) of the atoms in our MD simulation. As we briefly discussed in our lectures, the RDF gives a quantitative description of the connectivity of atoms in our material, giving the average distances between atoms of different element pairs.

To do this, we can use the `Analysis` class from ASE:

In [ ]:
from ase.geometry.analysis import Analysis

analysis = Analysis(traj)
rdf_per_timestep = analysis.get_rdf(rmax=5, nbins = 25)  # specify radial cutoff for RDF, and num histogram bins from 0 to `rmax`

In [ ]:
# Plot the average RDF:
import matplotlib.pyplot as plt
import numpy as np

r_bins = np.linspace(0, 5, 25)  # from 0 to rmax, with nbins intervals

# this rdf data includes all timesteps, so remove the equilibration period and only include the production stage:
production_stage_rdf_per_timestep = rdf_per_timestep[250:]  # only from 250th timestep on

# get average RDF over this time period:
production_stage_average_rdf = np.mean(production_stage_rdf_per_timestep, axis=0)

# plot
fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(r_bins, production_stage_average_rdf, color="steelblue")
ax.set_xlabel(r"Distance $r$ ($\mathrm{\AA}$)")
ax.set_ylabel("Average Radial Distribution Function (RDF)")
plt.show()


This plot is a bit jagged. Why? How can we get a smoother plot here?

This is the total (non-element-specific) RDF for all atoms in the simulation here. Now let's plot the RDF for some specific elemental pairs; namely Li-Li, P-S and Li-S:

In [ ]:
# get Li-Li, P-S and Li-S RDFs, and plot separately

Li_Li_rdf_per_timestep = analysis.get_rdf(rmax=5, nbins = 200, elements=['Li'])
P_S_rdf_per_timestep = analysis.get_rdf(rmax=5, nbins = 200, elements=['P', 'S'])
Li_S_rdf_per_timestep = analysis.get_rdf(rmax=5, nbins = 200, elements=['Li', 'S'])
r_bins = np.linspace(0, 5, 200)

Li_Li_production_stage_average_rdf = np.mean(Li_Li_rdf_per_timestep[200:], axis=0)
P_S_production_stage_average_rdf = np.mean(P_S_rdf_per_timestep[200:], axis=0)
Li_S_production_stage_average_rdf = np.mean(Li_S_rdf_per_timestep[200:], axis=0)

Now let's plot these RDFs all alongside:

In [ ]:
f,ax = plt.subplots(figsize=(7,5))
ax.plot(r_bins, Li_Li_production_stage_average_rdf, color="steelblue", label="Li-Li")
ax.plot(r_bins, P_S_production_stage_average_rdf, color="green", label="P-S")
ax.plot(r_bins, Li_S_production_stage_average_rdf, color="purple", label="Li-S")
ax.set_xlabel(r"Distance $r$ ($\mathrm{\AA}$)")
ax.set_ylabel("Average Radial Distribution Function (RDF)")
ax.legend()
plt.show()

### Questions — Part 3
- From our visualisations, which atoms are moving the most in the simulation? Does this match our expectations? Why?
- How do we know if the simulation has properly equilibrated?
- What would happen if you used a timestep of 25 fs instead of 1 fs? Why?
- From the RDFs how crystalline/disordered/liquid-like is our material (and specific element pairings) in this simulation? Does this match our expectations?

---
# Part 4: Extracting the Diffusion Coefficient

As very briefly mentioned in lectures, the **mean squared displacement (MSD)** quantifies how far atoms have moved from
their initial positions, averaged over all atoms of that type:

$$\text{MSD}(\Delta t) = \left\langle \frac{1}{N_{\text{Li}}} \sum_{i=1}^{N_{\text{Li}}} |\mathbf{r}_i(\Delta t) - \mathbf{r}_i(t=0)|^2 \right\rangle_{\Delta t}$$

Where $\Delta t$ is the time change, $N_{\text{Li}}$ is the number of Li atoms and $\mathbf{r}_i(\Delta t)$ is the position of the $i$th Li atom at time $\Delta t$.

For a diffusing species in 3D, the MSD grows linearly with time according to:

$$\text{MSD}(\Delta t) = 6D\Delta t$$

where $D$ is the **self-diffusion coefficient**. The factor of 6 comes from the three
spatial dimensions (2 per dimension).

We can use the `DiffusionCoefficient` class from ASE to directly compute the diffusion coefficient from the trajectory -- this is a nice example of the benefits of the computational materials science ecosystem! Complex analysis code has already been written for us, so we just need to feed in the right data to proceed with our material simulations.

In [ ]:
from ase.md.analysis import DiffusionCoefficient

msd_calc = DiffusionCoefficient(
    traj,  # trajectory over which to compute the diffusion coefficient
    timestep=1.0,  # timestep used with this trajectory
    atom_indices=list(range(27))  # atomic indices for which to compute diffusion coefficients (the first 27 atoms are Li, in our Li27P12S44 cell)
)  
msd_calc.calculate()  # run the Diffusion Coefficient calculation


In [ ]:
msd_calc.plot()

Something is a bit inaccurate with how we have performed extracted the diffusion coefficient from our MD trajectory here – what? Let's fix it and re-run the diffusion coefficient analysis.

We used the whole trajectory for the diffusion analysis, but should only use the production stage!

In [ ]:
from ase.md.analysis import DiffusionCoefficient

msd_calc = DiffusionCoefficient(
    # this selects only the 250th timestep onwards of the trajectory (i.e. the production stage)
    traj[250:],  # trajectory over which to compute the diffusion coefficient
    timestep=1.0,  # timestep used with this trajectory
    atom_indices=list(range(27))  # atomic indices for which to compute diffusion coefficients (the first 27 atoms are Li, in our Li27P12S44 cell)
)  
msd_calc.calculate()  # run the Diffusion Coefficient calculation

msd_calc.plot()

In [ ]:
# we can then get the fitted diffusion coefficient from the linear fit with this function:
D_array, std_dev_array = msd_calc.get_diffusion_coefficients()
D = D_array[0]*units.fs  # units are Å^2/fs
D_cm2s = D * 0.1  # Å^2/fs = 0.1 cm^2/s
std_dev = std_dev_array[0]*units.fs  # units are Å^2/fs
std_dev_cm2s = std_dev * 0.1  # Å^2/fs = 0.1 cm^2/s
print(f"Linear fit diffusion coefficient: {D_cm2s:.2e} (+/- {std_dev_cm2s:.0e}) cm²/s")

### Questions — Part 4

- Does the MSD show a clear linear regime? Why/why not? What does this tell us about the stability/convergence of our simulation?
  - Is it more ballistic ($\propto t^2$) or diffusive ($\propto t$) behaviour?
- How sensitive is the extracted $D$ to the choice of fitting range?

- Why do we only compute the MSD for Li atoms and not for P or S?

- How does our extracted diffusion coefficient $D$ compare to typical diffusion coefficients? (Based on Googling and quickly searching the literature (Elicit, SciSpace and Consensus are some popular LLM literature search tools...))

- What about for (defective) Li solid-state electrolytes, under our specific simulation conditions?

- Would the  increase or decrease with the simulation temperature?

- Do we have enough data to get a good fit for our diffusion coefficient here? How does our standard deviation of the fit compare to the extracted value for the diffusion coefficient?

**Not in class notebook:**
- Experimental measurements for Li₇P₃S₁₁ get $D \sim 10^{-8}$–$10^{-7}$ cm²/s at room temperature. How does our value of $D$ differ and why?

---
# Summary

In this practical you have:

1. **Explored** a DFT dataset for a (defective) Li₇P₃S₁₁ solid electrolyte
2. **Trained** a MACE machine-learning interatomic potential on DFT configurations
3. **Ran** molecular dynamics simulations of this solid electrolyte
4. **Extracted** the Li-ion self-diffusion coefficient from mean squared displacement analysis

### The big picture

This workflow — DFT → MLIP → MD → transport properties — is now a standard tool in
computational materials science. The same approach can be applied to simulations of:
- Other solid electrolytes (garnet, perovskite, NASICON-type stryctures)
- Electrode materials (lithium intercalation kinetics)
- Thermal conductivity
- Phase transitions and structural dynamics
- ...

Here we have seen that Li-ion diffusion is indeed very fast in this system, corroborating the fact that it is a promising candidate for a solid-state electrolyte for advanced Li-ion batteries!

**Reminder:**

In this practical there are a number of technical and discussion/conceptual questions. It is recommended that you record answers as you go, and download the completed notebook at the end of the practical session, to use as a resource for reference with the practical report and follow-on task.

---
# Part 5: Determining the Activation Energy

Li-ion diffusion is a thermally activated process which can described by the **Arrhenius relation**:

$$D(T) = D_0 \exp\left(-\frac{E_a}{k_B T}\right)$$

where:
- $D_0$ is the pre-exponential factor
- $E_a$ is the **activation energy** (the barrier for Li-ion hopping)
- $k_B$ is Boltzmann's constant
- $T$ is the temperature

How can we use this relation and our ability to compute $D$ with MD to determine the activation energy?

Do this, determining $D_0$, $E_a$ for Li-ion diffusion in this system.


Here are some cells to run which setup our data, so we don't need to re-run the whole notebook above:

In [ ]:
# this automatically downloads our data, so we don't need to manually download and upload
# as we did in the practical session:
!git clone https://github.com/SAM-Laboratory/AMET_Comp_Practical
%cd AMET_Comp_Practical
!gzip -d LiPS.xyz.gz  # decompress the data

In [ ]:
# Install required packages:
!pip install mace-torch ase matplotlib numpy scipy pymatviz pymatgen
!pip install cuequivariance_torch  # this speeds up our model training (accelerated tensor products)
# Make sure Runtime -> Change runtime type -> GPU is selected

In [ ]:
# import standard scientific Python packages; numpy, matplotlib:
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter("ignore")  # ignore irrelevant warnings for this practical

In [ ]:
# Load the dataset, which is stored in XYZ format (.xyz)
# Each frame contains: atomic positions, forces, total energy, and the simulation cell
from ase.io import read

print("Loading DFT dataset...")
dataset = read("LiPS.xyz", index=":")  # read all frames in the dataset
print(f"Loaded {len(dataset)} configurations")

In [ ]:
from ase.io import write
from sklearn.model_selection import train_test_split

# first, subsample 10% of the dataset
subsampled_dataset = dataset[::10]  # this takes every 10th frame in the dataset

# First split off train from test/val; with 80/20 split
train_set, test_val_set = train_test_split(subsampled_dataset, train_size=0.8, test_size=0.2)

# split test/val into val and test; with 50/50 split (corresponding to 10/10 split overall)
test_set, val_set = train_test_split(test_val_set, test_size=0.5, train_size=0.5)

# Save split datasets to files for MLIP training
write("LiPS_train.xyz", train_set)
write("LiPS_val.xyz", val_set)
write("LiPS_test.xyz", test_set)

print(f"Training set:   {len(train_set)} configurations")
print(f"Validation set: {len(val_set)} configurations")
print(f"Test set:       {len(test_set)} configurations")

In [ ]:
# here we define a function to run a simple MD simulation, which uses functions from the ASE package:
from ase import units
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

import random
import os
import pylab as pl
from IPython import display

def simpleMD(atoms_init, temp, calc, output_fname, timestep, num_steps):
    # set our calculator:
    atoms_init.set_calculator(calc)

    # initialize the temperature
    random.seed(701)  # (this is just to make the MD simulation fully reproducible)
    MaxwellBoltzmannDistribution(atoms_init, temperature_K=temp)  # initialize temperature at `temp`

    # set up our MD dynamics; here we use a 'Langevin' thermostat,
    # which is a simple way to control the temperature of the system.
    # we also set the timestep here
    dyn = Langevin(atoms_init, timestep*units.fs, temperature_K=temp, friction=0.1)
    # see here: https://ase-lib.org/ase/md.html

    # here we add some code to monitor the temperature and energy of the system as
    # a function of time in the MD simulation:
    time_fs = []
    temperature = []
    energies = []

    os.system('rm -rfv '+output_fname)  # remove any previously stored trajectory with the same name
    fig, ax = pl.subplots(2, 1, figsize=(6,6), sharex='all', gridspec_kw={'hspace': 0, 'wspace': 0}, dpi=500)  # generates dynamic plot as MD proceeds

    def write_frame():  # this is a function we add to the MD dynamics to monitor the system
        dyn.atoms.write(output_fname, append=True)  # store current frame in output trajectory file
        time_fs.append(dyn.get_time()/units.fs)  # store the current time in fs
        temperature.append(dyn.atoms.get_temperature())  # store the current temperature
        energies.append(dyn.atoms.get_potential_energy()/len(dyn.atoms))  # store the current energy

        # plot the energy of the system as a function of time
        ax[0].plot(np.array(time_fs), np.array(energies), color="b")
        ax[0].set_ylabel('E (eV/atom)')

        # plot the temperature of the system as a function of time
        ax[1].plot(np.array(time_fs), temperature, color="r")
        ax[1].set_ylabel('T (K)')
        ax[1].set_xlabel('Time (fs)')

        display.clear_output(wait=True)
        display.display(pl.gcf())

    dyn.attach(write_frame)  # adds our writing & plotting function to the MD calculation
    dyn.run(num_steps)  # runs our MD calculation

### Questions — Part 5

- How well converged are the $D_0$ and $E_a$ values here?
- Are the $D_0$ and $E_a$ values temperature-dependent? Why / why not?
- Does the Arrhenius plot show a clear linear relationship? Why / why not, and what does this imply about the diffusion mechanism?
- How does your activation energy of Li-ion diffusion compare to values in the literature for (non-defective) Li₇P₃S₁₁? If it differs, what are possible reasons?
- Would you trust the diffusion coefficient extracted at a very low temperature (e.g. 50 K) here? Why or why not?
- How would you improve the accuracy of the activation energy determination?

---
# Part 6: Learning Curves

A common and very useful task in MLIP training is to generate and consider 'learning curves', which plot the accuracy of the MLIP as a function of some parameter; typically either the size of the training dataset, the number of training epochs, or the number of parameters in the model.

This can give useful information about the model 'scaling' – i.e. how we can expect the accuracy of the model to improve with more data, more training, or more parameters.

Generate a learning curve for the MACE model here, with dataset size as our variable parameter, using datasets of every 1000th, 100th, 50th, 10th, 5th, 2nd and every frame in the AIMD dataset, as the dataset sizes to test – in the example below we take every 10th frame. For each, you will need to record the **_test_** RMSE and MAE of the model for energies and forces, to then plot against dataset size to generate the learning curve.

For model training here, you can reduce the number of training epochs to 20 to speed up the training process.

In [ ]:
# earlier we used this code to take every 10th frame from the raw AIMD dataset, to then
# generate training, validation and test sets:
from ase.io import write
from sklearn.model_selection import train_test_split

# this takes every 10th frame in the dataset; we can modify this to generate datasets
# of different sizes, and then train MACE models (with the same model settings, for consistency)
# on these different datasets to generate learning curves:
subsampled_dataset = dataset[::10]  # this takes every 10th frame in the dataset

# First split off train from test/val; with 80/20 split
train_set, test_val_set = train_test_split(subsampled_dataset, train_size=0.8, test_size=0.2)

# split test/val into val and test; with 50/50 split (corresponding to 10/10 split overall)
test_set, val_set = train_test_split(test_val_set, test_size=0.5, train_size=0.5)

# Save split datasets to files for MLIP training
write("LiPS_train.xyz", train_set)
write("LiPS_val.xyz", val_set)
write("LiPS_test.xyz", test_set)

print(f"Training set:   {len(train_set)} configurations")
print(f"Validation set: {len(val_set)} configurations")
print(f"Test set:       {len(test_set)} configurations")

In [ ]:
# model training for different dataset sizes...

In [ ]:
# example code for plotting learning curves:
import matplotlib.pyplot as plt
import numpy as np

# 25001 is the number of frames in the AIMD dataset, so dataset sizes are 25001/[interval between frames extracted for training]
dataset_sizes = 25001/np.array([1000, 100, 50, 10, 5, 2, 1])  

per_atom_energy_MAEs = [7, 6, 5, 4, 3, 2, 1]  # example MAE values for different dataset sizes
per_atom_energy_RMSEs = [10, 8, 6, 4, 2, 1, 0.5]  # example RMSE values for different dataset sizes

force_MAEs = [0.05, 0.04, 0.03, 0.02, 0.01, 0.005, 0.001]  # example MAE values for different dataset sizes
force_RMSEs = [0.1, 0.08, 0.06, 0.04, 0.02, 0.01, 0.005]  # example RMSE values for different dataset sizes

# plot the learning curves:
f,ax = plt.subplots(figsize=(7,5))
ax.plot(dataset_sizes, per_atom_energy_MAEs, color="steelblue", label="Energy MAE")
ax.plot(dataset_sizes, per_atom_energy_RMSEs, color="steelblue", label="Energy RMSE")
ax.plot(dataset_sizes, force_MAEs, color="green", label="Force MAE")
ax.plot(dataset_sizes, force_RMSEs, color="green", label="Force RMSE")
ax.set_xlabel("Dataset Size")
ax.set_ylabel("MAE")
ax.legend()
# ax.set_xscale("log")  # uncomment these to see log-log plot, usually best!
# ax.set_yscale("log")
plt.show()

- What relationship do we find between the MAE/RMSEs and dataset size? (Linear, parabolic, exponential, logarithmic, etc.)
- What dataset size (assuming the same data origin, sampling technique and model settings as above) would we need to achieve:
    - Energy MAE < 10 meV/atom? <1 meV/atom? <0.1 meV/atom?
    - Force MAE < 50 meV/Å? <1 meV/Å?
    - Energy RMSE < 10 meV/atom? <0.1 meV/atom?
    - Force RMSE < 100 meV/Å? <10 meV/Å?

We can similarly generate a learning curve with _model size_ as our variable parameter, by varying the number of parameters in the model through the settings in our config file.

Generate such a learning curve, by varying the model hyperparameters in the config file and recording the MAE/RMSEs and the total number of model parameters (printed in the MACE training output log).
I recommend varying `num_channels`, `max_L`, and `max_ell` for this, but you can also vary other hyper-parameters too if you are feeling adventurous. Either way, you should aim for at least 5-6 datapoints (of different model sizes) to plot for the learning curve.

In [ ]:
# example code for plotting learning curves:
import matplotlib.pyplot as plt
import numpy as np

# total parameter counts of the models tested
model_sizes = [100000, 50000, 25000, 10000, 5000, 2500, 1000]

per_atom_energy_MAEs = [7, 6, 5, 4, 3, 2, 1]  # example MAE values for different model sizes
per_atom_energy_RMSEs = [10, 8, 6, 4, 2, 1, 0.5]  # example RMSE values for different model sizes

force_MAEs = [0.05, 0.04, 0.03, 0.02, 0.01, 0.005, 0.001]  # example MAE values for different model sizes
force_RMSEs = [0.1, 0.08, 0.06, 0.04, 0.02, 0.01, 0.005]  # example RMSE values for different model sizes

# plot the learning curves:
f,ax = plt.subplots(figsize=(7,5))
ax.plot(model_sizes, per_atom_energy_MAEs, color="steelblue", label="Energy MAE")
ax.plot(model_sizes, per_atom_energy_RMSEs, color="steelblue", label="Energy RMSE")
ax.plot(model_sizes, force_MAEs, color="green", label="Force MAE")
ax.plot(model_sizes, force_RMSEs, color="green", label="Force RMSE")
ax.set_xlabel("Model Size")
ax.set_ylabel("MAE")
ax.legend()
# ax.set_xscale("log")  # uncomment these to see log-log plot, usually best!
# ax.set_yscale("log")
plt.show()

- Is the scaling behaviour here similar to the dataset size learning curve, or not? Why?
- What happens to the time required to train (and evaluate) the model as we increase the model size?
- Why don't we just use the model settings which give us the most accurate model possible? (Thinking in terms of both training and production simulations)
- Roughly what model size do we need to achieve:
    - a force MAE < 10 meV/Å?
    - an energy MAE < 1 meV/atom?
    - a force RMSE < 10 meV/Å?
    - an energy RMSE < 0.1 meV/atom?

### Re-Evaluating the RDF and Diffusion Coefficient $D$

Re-compute the RDF and diffusion coefficient $D$ with one of the more accurate models obtained from the learning curves above (you will need to factor in evaluation speeds for this choice), with the goal of maximising the accuracy of these predicted properties.

- Explain your choice of model/dataset size for this evaluation.
- How do these values compare to the values obtained from that in the practical session? Is this reasonable? Which values would you trust more?
    - What does this tell us about the sensitivities of these particular properties to the accuracy of our underlying MD calculator?

### Data Sampling

Re-do the dataset-size learning curve, but instead of our interval-based sampling (of taking each n-th frame from the AIMD data), lets just take the first $n$ frames from the AIMD data.

In [ ]:
# Example code for taking the first n frames from the AIMD data:
from ase.io import write
from sklearn.model_selection import train_test_split

# this takes the first 500 frames in the dataset; we can modify this to generate datasets
# of different sizes, and then train MACE models (with the same model settings, for consistency)
# on these different datasets to generate learning curves:
subsampled_dataset = dataset[:500]  # this takes the first 500 frames in the dataset

# First split off train from test/val; with 80/20 split
train_set, test_val_set = train_test_split(subsampled_dataset, train_size=0.8, test_size=0.2)

# split test/val into val and test; with 50/50 split (corresponding to 10/10 split overall)
test_set, val_set = train_test_split(test_val_set, test_size=0.5, train_size=0.5)

# Save split datasets to files for MLIP training
write("LiPS_train.xyz", train_set)
write("LiPS_val.xyz", val_set)
write("LiPS_test.xyz", test_set)

print(f"Training set:   {len(train_set)} configurations")
print(f"Validation set: {len(val_set)} configurations")
print(f"Test set:       {len(test_set)} configurations")

- How does the learning curve change? Why? (think back to our lectures!)

As we revealed from our data analyses in the practical session, our training data here comes from a ab-initio MD simulations (i.e. MD with DFT as the calculator). 

- If we want to increase our dataset size to improve the accuracy of our trained MLIPs for this system, but for the minimal compute cost (i.e. minimum number of DFT calculations), what would be some possible strategies?

- What is the advantage/limitation of just running more AIMD simulations to extend the dataset?

### Notes on improving the MACE model:

MACE has different hyper parameters that controls the degree of accuracy and expressivity of the model. Setting these parameters is a trade off between accuracy and computational cost. Most MACE parameters have robust and well tested defaults. We call them here "Parameters to keep to default values" but mention them for general knowledge. The "Hyper Parameters to Change" are hyper parameters that need to be adjusted as they can be task or system dependent.

### Hyper Parameters to Change:

- **`--num_channels`: Number of channels**
  
  Determines the size of the model. The recommended value is `--num_channels=128` but other potential values are `64` for a faster model or `256` for a large but more accurate model.
  
  
- **`max_L`: Symmetry of the messages**

  Determines the symmetry of the messages. A value of `--max_L=0` means MACE will pass only invariant information between neigborhoods. It is the parameter **that affects the most the computational speed and the accuracy of the model**. `--max_L=0` are the fastest model, use them to train cheap model to run large and long simulations. `--max_L=1` is the default value, it is a good compromise between speed and accuracy. It is recommended to start with that value for a new project. `--max_L=2` are the most accurate models, use them to train very accurate but slower models.
  

- **`--r_max`: The cutoff radius**
  
  The cutoff used to create the local environment in each layer. `r_max=5.0` means atoms separated by a distance of more than 5.0 Å do not directly communicate in a single layer. When the model has multiple message-passing layers, atoms further than 5.0 Å can still communicate through later messages if intermediate proxy atoms exist. The effective receptive field of the model is `num_interactions * r_max`. The larger the `r_max`, the slower the model will be. It is recommended to use values between 4.0 Å and 7.0 Å.

---

### Hyper Parameters to keep to default values (for general knowledge):

- **`--num_interactions`: Message-passing layers**
  
  Controls the number of message-passing layers in the model. It should always be 2, and it is recommended not to modify it.


- **`--correlation`: The order of the many-body expansion**

  The body order that MACE induces at each layer. Choosing `--correlation=3` will create basis functions of up to 4-body (ijkl) indices, for each layer. If the model has multiple layers, the effective correlation order is higher. For example, a two-layer MACE with `--correlation=3` has an effective body order of 13.


- **`--max_ell`: Angular resolution**

  The angular resolution describes how well the model can describe angles. This is controlled by `max_ell` of the spherical harmonics basis (not to be confused with `max_L`). Larger values will result in more accurate but slower models. The default is `max_ell=3`, which is appropriate in most cases.
